In [0]:
import pandas as pd
from string import punctuation
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Part B

In [0]:
tweets = pd.read_csv('6ktweets.csv')

def remove_punctuation(item):
    for p in punctuation:
        item = item.lstrip().replace(p,'')
    return item

def conv_lowercase(x):
    return x.lower()

tweets['clean_tweets'] = tweets['tweets'].apply(remove_punctuation).apply(conv_lowercase)
#tweets

In [0]:
nltk.download()
tweets['word_freq_list'] = tweets['clean_tweets'].apply(word_tokenize).apply(set).apply(list)

# Remove stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 

tweets['word_freq_list'] = tweets['word_freq_list'].apply(remove_stopwords)

# Creating a list of all the words in the comments to count word frequency in the next step
count = []
for i in range(len(tweets)):
    count+=tweets['word_freq_list'][i]

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [0]:
# fetching the frequency of words
from nltk import FreqDist
word_freq = nltk.FreqDist(count)

In [0]:
#identified key issues

#trump issues
immigration = ['mexican', 'wall', 'immigrate', 'mexico', 'illegals', 'immigrants', 'immigrant']
impeach = ['impeachment', 'impeached', 'trumpimpeachment', 'impeachtrump']
foreign_relations = ['ukraine', 'isis', 'syria', 'china' ,'iran']

#biden issues
economy = ['job', 'jobs', 'money', 'taxes', 'taxpayer', 'economys', 'trumpeconomy']
education = [ 'college', 'school' ,'educate' ,'students' , 'student']
gun_control = ['control','weapons', 'weapon','guns', 'gun']

#replacements for trump and biden
trump = ['donaldtrump', 'realdonaldtrump',  'trump2020','trumps', 'donald','realtrump','MAGA','KAG','Trump']
biden = ['joebiden', 'joes','bidens', 'joe','biden']

attributes = immigration + impeach+ foreign_relations + economy + trump + biden +education + gun_control

In [0]:
#replacing words with key issues
key_issues = pd.DataFrame()
key_issues['attribute'] = attributes
key_issues['general_attribute'] = ''

key_issues['general_attribute'][0:len(immigration)] = 'immigration'
key_issues['general_attribute'][len(immigration):len(immigration)+len(impeach)] = 'impeach'
key_issues['general_attribute'][len(immigration)+len(impeach):len(immigration)+len(impeach)+len(foreign_relations)] = 'foreign_relations'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)] = 'economy'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+ len(economy):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)] = 'trump'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+len(biden)] = 'biden'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+ len(biden):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+len(biden) + len(education)] = 'education'
key_issues['general_attribute'][len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+ len(biden)+ len(gun_control):len(immigration)+len(impeach)+len(foreign_relations)+len(economy)+len(trump)+len(biden) + len(education)+len(gun_control)] = 'gun_control'
#key_issues

In [0]:
def word_to_attributes(s):
    s = " ".join(str(x) for x in s)
    for i in key_issues.index.values:
        s = s.replace(key_issues["attribute"][i].lower(),key_issues["general_attribute"][i].lower())
    return s

tweets['comments_attributes_replace'] = tweets['word_freq_list'].apply(word_to_attributes)

### C.To isolate a candidate and an issue, you have to write a parser that selects tweets that mention a candidate, takes a window around an issue, and chops off everything else, as shown in class.  

In [0]:
#making list of tweets that mention trump and list of tweets that mention biden
all_ = tweets['comments_attributes_replace'].apply(word_tokenize).apply(set).apply(list)
all_ = all_.tolist()
t =[]
b =[]
for i in all_: 
    for j in set(i):
        if j == 'trump':
            t.append(i)
        elif j == 'biden':
            b.append(i)             

In [0]:
#calculating total no of tweets that mention each of these key issues
economy =[]
immigration =[]
foreign_relations=[]
impeach=[]
for j in all_:
    for k in j:
        if k == 'economy':
            economy.append(j)
        if k == 'immigration':
            immigration.append(j)
        if k =='foreign_relations':
            foreign_relations.append(j)
        if k == 'impeach':
            impeach.append(j) 

In [0]:
#calculating total number of tweets that mention both key issues and trump
economy_trump =[]
immigration_trump =[]
foreign_relations_trump=[]
impeach_trump =[]

for j in t:
    for k in j:
        if k == 'economy':
            economy_trump.append(j)
        if k == 'immigration':
            immigration_trump.append(j)
        if k =='foreign_relations':
            foreign_relations_trump.append(j)
        if k == 'impeach':
            impeach_trump.append(j)           

In [0]:
#calculating total number of tweets that mention both key issues and biden
economy_biden =[]
immigration_biden =[]
foreign_relations_biden=[]
impeach_biden =[]

for c in b:
    for b1 in set(c):
        if b1 == 'economy':
            economy_biden.append(c)
            continue;
        if b1 == 'immigration':
            immigration_biden.append(c)
            continue; 
        if b1 =='foreign_relations':
            foreign_relations_biden.append(c) 
            continue;
        if b1 == 'impeach':
            impeach_biden.append(c)  
            

In [0]:
#lift analysis for trump and biden

#trump
lift_economy_trump = (len(tweets)*len(economy_trump))/(len(t)*len(economy))
lift_immigration_trump = (len(tweets)*len(immigration_trump))/(len(t)*len(immigration))
lift_foriegn_relations_trump = (len(tweets)*len(foreign_relations_trump))/(len(t)*len(foreign_relations))
lift_impeach_trump = (len(tweets)*len(impeach_trump))/(len(t)*len(impeach))

#biden
lift_economy_biden = (len(tweets)*len(economy_biden))/(len(b)*len(economy))
lift_immigration_biden = (len(tweets)*len(immigration_biden))/(len(b)*len(immigration))
lift_foriegn_relations_biden = (len(tweets)*len(foreign_relations_biden))/(len(b)*len(foreign_relations))
lift_impeach_biden = (len(tweets)*len(impeach_biden))/(len(b)*len(impeach))

In [0]:
data = [[lift_economy_trump, lift_immigration_trump, lift_foriegn_relations_trump, lift_impeach_trump], [lift_economy_biden, lift_immigration_biden, lift_foriegn_relations_biden,lift_impeach_biden]]
lift = pd.DataFrame(data, columns = ['Economy', 'Immigration', 'Foreign Relations', 'Impeach'])
lift.rename(index = {0: "Trump", 1:"Biden"},  inplace = True) 

In [0]:
lift

,Economy,Immigration,Foreign Relations,Impeach
Trump,0.918829,0.995025,1.188502,1.293532
Biden,1.794325,0.622407,2.489627,0.398340


In [0]:
#forming window around tweets that mention trump and issues

from nltk.tokenize.treebank import TreebankWordDetokenizer
impeach_sentiment_trump=[]
economy_sentiment_trump=[]
immigration_sentiment_trump=[]
foreign_relations_sentiment_trump=[]
for s in impeach_trump:
    for i in range(len(s)):
        if s[i] =='impeach':
            impeach_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))
            
for s in economy_trump:
    for i in range(len(s)):
        if s[i] =='economy':
            economy_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in immigration_trump:
    for i in range(len(s)):
        if s[i] =='immigration':
            immigration_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in impeach_trump:
    for i in range(len(s)):
        if s[i] =='foreign_relations':
            foreign_relations_sentiment_trump.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

In [0]:
#forming window around tweets that mention biden and issues
impeach_sentiment_biden=[]
economy_sentiment_biden=[]
immigration_sentiment_biden=[]
foreign_relations_sentiment_biden=[]
for s in impeach_biden:
    for i in range(len(s)):
        if s[i] =='impeach':
            impeach_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))
            
for s in economy_biden:
    for i in range(len(s)):
        if s[i] =='economy':
            economy_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in immigration_biden:
    for i in range(len(s)):
        if s[i] =='immigration':
            immigration_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

for s in foreign_relations_biden:
    for i in range(len(s)):
        if s[i] =='foreign_relations':
            foreign_relations_sentiment_biden.append(TreebankWordDetokenizer().detokenize(s[i-2:i+2]))

In [0]:
#performing sentiment analysis 

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

def sentiment_analysis(list_):

    sia = SIA()
    r = []

    for i in list_:
        score_ = sia.polarity_scores(i)
        r.append(score_)
    sum_=0
    for i in range(len(r)):
        sum_ = sum_ + r[i]['pos']- r[i]['neg']
    return sum_ 

In [0]:
#getting lift scores for all key issues for each candidate

impeach_sentiment_t= sentiment_analysis(impeach_sentiment_trump)
economy_sentiment_t= sentiment_analysis(economy_sentiment_trump)
immigration_sentiment_t =sentiment_analysis(immigration_sentiment_trump)
foreign_relations_sentiment_t= sentiment_analysis(foreign_relations_sentiment_trump)

impeach_sentiment_b = sentiment_analysis(impeach_sentiment_biden)
economy_sentiment_b= sentiment_analysis(economy_sentiment_biden)
immigration_sentiment_b =sentiment_analysis(immigration_sentiment_biden)
foreign_relations_sentiment_b= sentiment_analysis(foreign_relations_sentiment_biden)

In [0]:
data = [[economy_sentiment_t, immigration_sentiment_t, foreign_relations_sentiment_t, impeach_sentiment_t], [economy_sentiment_b, immigration_sentiment_b, foreign_relations_sentiment_b,impeach_sentiment_b]]
sentiment = pd.DataFrame(data, columns = ['Economy', 'Immigration', 'Foreign Relations', 'Impeach'])
sentiment.rename(index = {0: "Trump", 1:"Biden"},  inplace = True) 

In [0]:
sentiment

,Economy,Immigration,Foreign Relations,Impeach
Trump,0.720,1.235,0.455,0.324
Biden,-0.401,-0.538,-1.266,0.444


### Part E

In [0]:
## get top ten cities of each state
Michigan = ['Detroit','Grand Rapids','Warren', 'Sterling Heights','Ann Arbor','Lansing','Flint','Dearborn','Livonia','Troy']
Pennsylvania = ['Philadelphia','Pittsburgh','Allentown','Erie','Reading','Upper Darby','Scranton','Bethlehem','Lancaster','Millcreek']
Wisconsin = ['Milwaukee','Madison','Green Bay','Kenosha','Racine','Appleton','Waukesha','Eau Claire','Oshkosh','Janesville']

In [0]:
## make the city name all lower case
mi = []
for x in Michigan:
    mi.append(x.lower())
    
pen = []
for x in Pennsylvania:
    pen.append(x.lower())
    
wi = []
for x in Wisconsin:
    wi.append(x.lower())

In [0]:
threestates = []
for x in tweets['User Location'].unique():
    if 'michigan' in str(x).lower() or 'pennsylvania'in str(x).lower() or 'wisconsin' in str(x).lower() or str(x).lower().endswith(', mi') or str(x).lower().endswith('pa') or str(x).lower().endswith('wi') or str(x).lower() in mi or str(x).lower() in pen or str(x).lower() in wi:
        threestates.append(x)

threestates

['Michigan, USA',
 'Pennsylvania',
 'Green Bay',
 'Pennsylvania, USA',
 'Verona WI',
 'Altoona PA',
 'Wisconsin, USA',
 'Lehigh Valley PA',
 'DELCO Phila, Pa',
 'Philadelphia, PA',
 'Eastern PA',
 'Warren, PA',
 'Butler, PA',
 'Philadelphia',
 'Somewhere in Pennsylvania!',
 'Little Chute, WI',
 'Easton, PA',
 'Holt, MI',
 'Grand Blanc, MI',
 'Pittsburgh, PA',
 'Tempe, AZ & Door County, WI',
 'Waterford, PA',
 'Lebanon, PA',
 'Monaca, PA',
 'Wisconsin, USA ',
 'Loganville, PA',
 'Michigan U.S.A.',
 'Near Pittsburgh PA',
 'Western Pa',
 'Detroit, MI',
 'Huntingdon Valley, PA',
 'Oakland County, Mi',
 'Cranberry Township, PA',
 'Hobart, WI',
 'Lancaster, PA',
 'Clinton, MI',
 'Southfield, MI',
 'Michigan',
 'Lock Haven, PA',
 'Ligonier, PA',
 'Davidsville, PA',
 'New York / Pennsylvania',
 'Mt. Pleasant Michigan',
 'Milwaukee',
 'Superior, WI',
 'Port Huron, MI',
 'PA']

In [0]:
for index, row in tweets.iterrows():
    if row['User Location'] in threestates:
        tweets.loc[index,'User Location'] = 'battleground'

In [0]:
data1 = tweets.copy()
data1[data1['User Location'] == 'battleground']

,Unnamed: 0,Date,tweets,User Location,hashtag,clean_tweets,word_freq_list,comments_attributes_replace
13,13,10/14/2019 0:35,@NBCNews Unfounded? You people are nuts! Oh ...,battleground,#MAGA2020,nbcnews unfounded you people are nuts oh wai...,"[youre, never, mind, wait, people, nuts, nbcne...",youre never mind wait people nuts nbcnews nbc ...
32,32,10/14/2019 0:24,@BillKristol Drunk again? #MAGA2020,battleground,#MAGA2020,billkristol drunk again maga2020,"[billkristol, drunk, maga2020]",billkristol drunk trump2020
54,54,10/14/2019 0:04,@DavidJHarrisJr @realDonaldTrump That would of...,battleground,#MAGA2020,davidjharrisjr realdonaldtrump that would of m...,"[better, realdonaldtrump, made, seeing, last, ...",better trump made seeing last hat trump week f...
155,155,10/13/2019 22:45,No matter how hard they try to hinder our abil...,battleground,#MAGA2020,no matter how hard they try to hinder our abil...,"[strong…, hard, positive, train, matter, hinde...",strong… hard positive train matter hinder http...
164,164,10/13/2019 22:38,@thehill @realDonaldTrump You don’t speak for ...,battleground,#MAGA2020,thehill realdonaldtrump you don’t speak for th...,"[speak, us, maga2020, public, trumpin, magats,...",speak us trump2020 public trumpin trumpts amer...
...,...,...,...,...,...,...,...,...
5888,5888,10/11/2019 0:47,"I agree, @charliekirk11 !#MAGA2020 https://t.c...",battleground,#MAGA2020,i agree charliekirk11 maga2020 httpstcovo4vppymfe,"[agree, charliekirk11, httpstcovo4vppymfe, mag...",agree charliekirk11 httpstcovo4vppymfe trump2020
5900,5900,10/11/2019 0:46,Minneapolis police officer running against Ilh...,battleground,#MAGA2020,minneapolis police officer running against ilh...,"[running, constituents, minneapolis, omar, ilh...",running constituents minneapolis omar ilhan ab...
5905,5905,10/11/2019 0:44,"David Bossie: Impeachment inquiry? No, America...",battleground,#MAGA2020,david bossie impeachment inquiry no americans ...,"[americans, bossie, conclusion, httpstcoadqt9o...",americans bossie conclusion httpstcoadqt9otcye...
5908,5908,10/11/2019 0:44,@SpeakerPelosi @RepAdamSchiff Fact: only way ...,battleground,#MAGA2020,speakerpelosi repadamschiff fact only way tru...,"[realdonaldtrump, speakerpelosi, way, fact, 20...",trump speakerpelosi way fact 2020 impeach reel...


In [0]:
data1 = data1.dropna().copy()
data1.loc[data1['User Location'] == 'battleground','tweets'] = 'battleground' + data1['tweets']
data1

,Unnamed: 0,Date,tweets,User Location,hashtag,clean_tweets,word_freq_list,comments_attributes_replace
0,0,10/14/2019 0:47,Why do they keep electing elites that have mil...,United States,#MAGA2020,why do they keep electing elites that have mil...,"[trump2020, maga2020…, millions, keep, themmag...",trump trump2020… millions keep themtrump dolla...
1,1,10/14/2019 0:46,No Shame! \nThanks @carlosrossimc for the vers...,United States,#MAGA2020,no shame \nthanks carlosrossimc for the verse\...,"[kag, kagchallenge, verse, trump2020, carlosro...",trump trumpchallenge verse trump carlosrossimc...
3,3,10/14/2019 0:45,@realDonaldTrump @FoxNews needs to dump Wallac...,"Boston, MA",#MAGA2020,realdonaldtrump foxnews needs to dump wallace ...,"[time, httpstcoaukdrtpmkc, every, wallace, swi...",time httpstcoaukdrtpmkc every immigrationace s...
4,4,10/14/2019 0:45,President Trump is 100% Correct on Syria Withd...,Florida of course!,#MAGA2020,president trump is 100 correct on syria withdr...,"[last, –, stable, ⁦potus⁩, genius🇺🇸🇺🇸…, refuge...",last – stable ⁦potus⁩ genius🇺🇸🇺🇸… refuge withd...
5,5,10/14/2019 0:44,@JoeBiden Poor Joe...#MAGA #MAGA2020,Corpus Christi,#MAGA2020,joebiden poor joemaga maga2020,"[maga2020, poor, joemaga, joebiden]",trump2020 poor bidentrump biden
...,...,...,...,...,...,...,...,...
5993,5993,10/11/2019 0:19,Thank you to all my friends for sharing pictur...,"Minnesota, USA",#MAGA2020,thank you to all my friends for sharing pictur...,"[sharing, videos, minnesota…, thank, ’, httpst...",sharing videos minnesota… thank ’ httpstcov83p...
5994,5994,10/11/2019 0:19,Thank you to all my friends for sharing pictur...,"Minnesota, USA",#MAGA2020,thank you to all my friends for sharing pictur...,"[sharing, videos, minnesota…, thank, ’, friend...",sharing videos minnesota… thank ’ friends pict...
5995,5995,10/11/2019 0:18,Trump’s children take in millions overseas as ...,"Portland, OR",#MAGA2020,trump’s children take in millions overseas as ...,"[overseas, son, president, httpstcof4j5mupjmb,...",overseas son president httpstcof4j5mupjmb trum...
5997,5997,10/11/2019 0:17,🚨BREAKING🚨\n#Whistleblower Worked With #JoeBid...,USA,#MAGA2020,🚨breaking🚨\nwhistleblower worked with joebiden...,"[httpstcox0cjtigzhe, joebidencorruption, whist...",httpstcox0cjtigzhe bidencorruption whistleblow...


In [0]:
people_trump = ['trump','battleground']
trump_df = pd.DataFrame(columns = people_trump)

def candidates_mentioned(item):
    if p.lower() in item.lower():
        return 1
    else:
        return 0
      
for p in people_trump:
    trump_df[p] = data1['tweets'].apply(candidates_mentioned)
    
    
    
people_dem = ['biden','battleground']
dem_df = pd.DataFrame(columns = people_dem)

def candidates_mentioned(item):
    if p.lower() in item.lower():
        return 1
    else:
        return 0
      
for p in people_dem:
    dem_df[p] = data1['tweets'].apply(candidates_mentioned)

In [0]:
liftratio1=pd.DataFrame(columns = people_trump)
for i in range(len(people_trump)):
    new_list = []
    for j in range(len(people_trump)):
        if (i!=j):
            numerator = ((trump_df[people_trump[i]] + trump_df[people_trump[j]]) > 1).sum()
            denominator = trump_df[people_trump[j]].sum()*trump_df[people_trump[i]].sum()
            lift = numerator*len(trump_df)/denominator
            liftratio1.loc[people_trump[i],people_trump[j]] = lift
print ('Below are the lift ratios among Trump and battleground')
liftratio1

Below are the lift ratios among Trump and battleground


,trump,battleground
trump,NaN,1.0759
battleground,1.0759,NaN


In [0]:
liftratio2=pd.DataFrame(columns = people_dem)
for i in range(len(people_dem)):
    new_list = []
    for j in range(len(people_dem)):
        if (i!=j):
            numerator = ((dem_df[people_dem[i]] + dem_df[people_dem[j]]) > 1).sum()
            denominator = dem_df[people_dem[j]].sum()*dem_df[people_dem[i]].sum()
            lift = numerator*len(dem_df)/denominator
            liftratio2.loc[people_dem[i],people_dem[j]] = lift
print ('Below are the lift ratios among Biden and battleground')
liftratio2

Below are the lift ratios among Biden and battleground


,biden,battleground
biden,NaN,1.886
battleground,1.886,NaN


## Lift analysis

In [0]:
liftdf = pd.DataFrame(index=['Battleground'],columns=['Trump','Biden'])
liftdf.iloc[0,0]=liftratio1.iloc[0,1]
liftdf.iloc[0,1]=liftratio2.iloc[0,1]
liftdf

,Trump,Biden
Battleground,1.0759,1.886


In [0]:
data2 = data1[data1['User Location'] == 'battleground']

In [0]:
#get the compound sentiment score of tweets mentioned Trump
trumpsenti=[]
analyzer = SentimentIntensityAnalyzer()
for index, row in data2.iterrows():
    if 'trump' in row['tweets'].lower():
        vs = analyzer.polarity_scores(row['tweets'])
        trumpsenti.append(vs['compound'])

In [0]:
#get the compound sentiment score of tweets mentioned Biden
demsenti=[]
analyzer = SentimentIntensityAnalyzer()
for index, row in data2.iterrows():
    if 'biden' in row['tweets'].lower():
        vs = analyzer.polarity_scores(row['tweets'])
        demsenti.append((vs['compound']))

In [0]:
# take the average semtiment score of each tweet
sentimentdf = pd.DataFrame(index=['Battleground'],columns=['Trump','Biden'])
sentimentdf.iloc[0,0]=np.mean(trumpsenti)
sentimentdf.iloc[0,1]=np.mean(demsenti)
sentimentdf

,Trump,Biden
Battleground,0.158118,0.0489545


In [0]:
# a more detailed report of people's sentiment when they talk about Trump
trumppos = 0
trumpneg = 0
for x in trumpsenti:
    if x>0:
        trumppos+=1
    if x<0:
        trumpneg+=1
trumppos = trumppos/len(trumpsenti)
trumpneg = trumpneg/len(trumpsenti)


print(str(trumppos*100) +"% of tweets have postive sentiments regarding to Trump in battleground.")
print(str((1-trumpneg-trumppos)*100) +"% of tweets have neutral sentiments regarding to Trump in battleground.")
print(str(trumpneg*100) +"% of tweets have negitive sentiments regarding to Trump in battleground.")

44.776119402985074% of tweets have postive sentiments regarding to Trump in battleground.
35.82089552238806% of tweets have negitive sentiments regarding to Trump in battleground.
19.402985074626866% of tweets have negitive sentiments regarding to Trump in battleground.


In [0]:
# a more detailed report of people's sentiment when they talk about Biden
bidenpos = 0
bidenneg = 0
for x in demsenti:
    if x>0:
        bidenpos+=1
    if x<0:
        bidenneg+=1
        
bidenpos = bidenpos/len(demsenti)
bidenpos = bidenneg/len(demsenti)


print(str(bidenpos*100) +"% of tweets have postive sentiments regarding to Biden in battleground.")
print(str((1-bidenpos-bidenpos)*100) +"% of tweets have neutral sentiments regarding to Biden in battleground.")
print(str(bidenpos*100) +"% of tweets have negitive sentiments regarding to Biden in battleground.")

27.27272727272727% of tweets have postive sentiments regarding to Biden in battleground.
45.45454545454546% of tweets have negitive sentiments regarding to Biden in battleground.
27.27272727272727% of tweets have negitive sentiments regarding to Biden in battleground.


# Part F

### According to the lift analysis, Trump is associated more with immigration and impeach issues than Biden. While Biden is more associated with economy and foreign relations issues. According to the sentiment analysis, people have very diverse sentiments of Trump and Biden on different issues. As for economy issue, people’s overall sentiment of Trump is positive. On the other hand, Biden is not favorable because the sentiment score is negative. For immigration issue, Biden is favorable, and Trump is not liked. As for foreign relations, both candidates are not favorable. However, people still prefer Trump compared to Biden. For impeach issue, Trump is more favorable than Biden. And people’ssentiment of Biden is neutral on impeach issue.Trump should improve his reputation on immigration and foreign relation issues, because on these issues Biden is favorable. Our recommendation is that he should not post too many twittes abouthis politics. Even he wants to post something on Twitter, he should not show the aggressiveness because it will make people dislike him. Especially for immigration and foreign relation issues, on whichpeople show negative sentiments. As for Biden, he should work on improving his recognition on solvingeconomy and impeach issues, since Trump is preferred compared to him on these two issues. What’s more, he should increase the frequency of posting twittes. Because Trump posts a lot, and it will shift the focus away from him.  According to the result of lift analysis Trump is not frequently discussed in Michigan, Pennsylvania and Wisconsin. The probability of Trump been discussed is less than random selection. Compared to Trump, Biden is much more likely to be discussed by people in Michigan, Pennsylvania and Wisconsin. The probability of Biden been discussed is significantly higher than random selection. According to the result of sentiment analysis, for people in Michigan, Pennsylvania and Wisconsin, theirsentiments about Trump is more positive than their sentiments about Biden. In other words, Trump is more favorable in Michigan, Pennsylvania and Wisconsin, compared to Biden. After combining the lift analysis and the sentiment analysis, we are confidence to conclude that although Biden is mentioned more in people’s discussions, it is likely that those mentions are close to neutral sentiments. Although Trump is mentioned less frequently, people in Michigan, Pennsylvania and Wisconsin like him more than Biden. Trump should give more speeches in Michigan, Pennsylvania and Wisconsin. Because he is favorable compared to Biden. By giving more speeches, people will discuss him more, and it is beneficial. Biden should participate more on Twitter and speak for people’s benefits; therefore, he would have a chance to have higher sentiment score than Trump.